# Bases de Datos: GEIH DICIEMBRE 2021

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

 ## Area-Caracteristicas Generales(Busca la variable de genero)

In [2]:
car = pd.read_csv(r'C:\Users\JORGE\Documents\CSV\Enero\Área - Características generales (Personas).csv', delimiter = ';', encoding='latin-1')


### Feauture Selection Area Caracteristicas Generales

In [3]:
# se seleccionan las variables que se van a dejar (Feature Selection)
car = car[["DIRECTORIO","ORDEN","HOGAR","P6071S1","P6040","P6020" ,"P6050","P6071","P6070","DPTO" ,"P6090","P6170","P6175","ESC"]]
# Se les cambia el nombre pra que sea facil trabajar
car.columns=["DIRECTORIO","ORDEN","HOGAR","orden_conyugue", "edad","sexo", "rol_hogar", "conyuge_en_hogar",  "estado_civil"  ,"DPTO","seguridadsocial","estudia","educ_publica","educacion"]

### Data manipulation caracteristicas Generales

In [4]:
car["sexo"]=car["sexo"].replace({1:'hombre',2:'mujer'})
car["rol_hogar"]=car["rol_hogar"].replace({1:'jefe',2:'pareja_jefe',3:'hijo/a_jefe',4:'nieto_jefe',5:'otro',6:'empleado_jefe',7:'pensionista',8:'trabajador',9:'otronopariente'})
car["seguridadsocial"]=car["seguridadsocial"].replace({1:'asegurado',2:'noasegurado'})
car["estudia"]=car["estudia"].replace({2:0})
car["educ_publica"]=car["educ_publica"].replace({2:0})
car['DPTO'] = car['DPTO'].replace({68:'Santander', 8:'Atlantico', 76:'Valle', 23:'Cordoba', 17:'Caldas',13:'Bolivar',5:'Antioquia',11:'Bogota',52:'Nariño',66:'Risaralda',50:'Meta',54:'Norte de Santander',73:'Tolima'})
car['estado_civil'] = car['estado_civil'].replace({2:'Nocasados_viven>2años', 1:'Nocasados_viven<2años', 3:'Casados',4:'separados',5:'viudo',6:'soltero'})


## Area-Ocupados

In [5]:
ocu = pd.read_csv(r'C:\Users\JORGE\Documents\CSV\Enero\Área - Ocupados.csv',delimiter=';', encoding='latin-1')

### Feauture Selection Area ocupados

In [6]:
ocu = ocu[["DIRECTORIO","ORDEN","INGLABO",'P6440','P6460','P6790','OCI','P6430']]
ocu.columns = ["DIRECTORIO","ORDEN","INGLABO", 'tipocontrato','terminocontrato','mesestrabajoultimoaño','ocupados','tipocupacion']

### Data manipulation 

In [7]:
ocu['tipocontrato'] = ocu['tipocontrato'].replace({1:'Verbal',2:'Escrito',3:'Noinfo'})
ocu['terminocontrato'] = ocu['terminocontrato'].replace({1:'indefinido',2:'fijo',3:'Noinfo'})
ocu['tipocupacion'] = ocu['tipocupacion'].replace({1:'empleadoempresa',2:'empleadogobierno',3:'empleadodomestico',4:'cuentapropia',5:'empleador',6:'familiarnoremu',7:'empleadonoremu',8:'jornalero',9:'otro'})

### Mezcla Base 1 y 2

In [8]:
car_ocu = car.merge(ocu, on = ['DIRECTORIO','ORDEN'])

# Area otras ayudas

In [9]:
dom = pd.read_csv(r'C:\Users\JORGE\Documents\CSV\Enero\Área - Otras actividades y ayudas en la semana.csv',delimiter=';')

In [10]:
dom = dom[['DIRECTORIO','ORDEN','P7480S3A1']]
dom.columns=['DIRECTORIO','ORDEN','horas_labores_domesticas']

### Merge

In [11]:
# Base final
merge1 = car_ocu.merge(dom, on = ['DIRECTORIO','ORDEN'])
#print('La base de datos tiene '+str(len(df))+' observaciones y tiene '+str(len(df.dropna()))+' quitando valores nulos')

# Area vivienda

In [12]:
viv = pd.read_csv(r'C:\Users\JORGE\Documents\CSV\Enero\Área - Vivienda y Hogares.csv',delimiter=';')

In [13]:
viv = viv[["DIRECTORIO","HOGAR",'P5000','P5210S3','P5210S4','P5210S2','P5210S5','P5210S7','P5210S16','P5210S21','P5210S22','P6008','P4000','P4030S1']]
viv.columns = ["DIRECTORIO","HOGAR",'#_cuartos_vivienda','internet','lavadora','television','nevera','estufa','computador','moto','carroparticular','totalpersonashogar','tipovivienda','energia']

#viv['tipovivienda']=viv['tipovivienda'].replace({1:'casa',2:'apto',3:'cuartoalqui',4:'cuartootro',5:'indigena',6:'otra'})
viv['internet']=viv['internet'].replace({2:0})
viv['lavadora']=viv['lavadora'].replace({2:0})
viv['television']=viv['television'].replace({2:0})
viv['nevera']=viv['nevera'].replace({2:0})
viv['estufa']=viv['estufa'].replace({2:0})
viv['computador']=viv['computador'].replace({2:0})
viv['moto']=viv['moto'].replace({2:0})
viv['carroparticular']=viv['carroparticular'].replace({2:0})
viv['energia']=viv['energia'].replace({2:0})

# BAse final

In [14]:
df = merge1.merge(viv, on = ['DIRECTORIO',"HOGAR"])

# Regresion lineal del logaritmo del salario

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [16]:
df1 = df.drop(['terminocontrato',"educ_publica","DIRECTORIO","ORDEN","HOGAR","orden_conyugue","conyuge_en_hogar","ocupados"],axis=1)


In [17]:
df1 = pd.get_dummies(df1, dtype=int)

In [18]:
# se quitan los salarios 0 para poder utilizar log
df1 = df1[df1['INGLABO']>0]
df1['educacion**2']=df1['educacion']**2

In [19]:
df1 = df1.dropna(how='any')

## Analisis de correlacion de variables con el logaritmo del salario

In [20]:
def correlacion(datos,variable=str):
    a = datos.corr().loc[variable, :].sort_values(ascending=False)
    z = [i for i in a if i >= 0.1] # se quieren ver solo correlaciones mayores a 0.1
    head = len(z)
    return a.head(head)
    

In [21]:
correlacion(df1,'INGLABO')

INGLABO                          1.000000
educacion**2                     0.470052
educacion                        0.408543
carroparticular                  0.364283
tipocupacion_empleadogobierno    0.306786
computador                       0.287030
internet                         0.222275
tipocontrato_Verbal              0.213507
lavadora                         0.195068
television                       0.177770
tipocupacion_empleador           0.161884
estado_civil_Casados             0.156220
#_cuartos_vivienda               0.138805
nevera                           0.121666
mesestrabajoultimoaño            0.115561
Name: INGLABO, dtype: float64

In [22]:
# se convierte al logaritmo del salario basandose en literatura economica
y = np.log(df1['INGLABO'].values)
# se arma X, se quitan algunas dummies que no tienen observaciones
X = df1.drop(['INGLABO','tipocupacion_familiarnoremu','tipocupacion_empleadonoremu','tipocupacion_empleadonoremu','tipocupacion_familiarnoremu','tipocupacion_otro'], axis=1).values

## Modelo regresion lineal normal

In [23]:
def regresion_lineal(X,y):
    # division de la base 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    #normalizacion de X
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # entrenamiento del modelo
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    # predicciones
    y_hat = lr.predict(X_test)
    # metricas
    rmse = np.sqrt(mean_squared_error(y_test,y_hat))
    mse = mean_squared_error(y_test,y_hat)
    r2 = r2_score(y_test,y_hat)
    return {'rmse':rmse, 'mse':mse, 'r2':r2, 'numero de observaciones de entrenamiento':len(y_train)}

In [24]:
modelo=regresion_lineal(X,y)
print(modelo)


{'rmse': 0.610304340234178, 'mse': 0.3724713877086753, 'r2': 0.4809610187469888, 'numero de observaciones de entrenamiento': 5666}


## Modelo ridge

In [25]:
from sklearn.linear_model import Ridge

In [26]:
def regresion_ridge(X,y,alpha):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    #normalizacion de X
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # modelo ridge
    l_ridge = Ridge(alpha).fit(X_train,y_train)
    # predicciones
    y_hat = l_ridge.predict(X_test)
    # metricas
    rmse = np.sqrt(mean_squared_error(y_test,y_hat))
    mse = mean_squared_error(y_test,y_hat)
    r2 = r2_score(y_test,y_hat)
    return {'rmse':rmse, 'mse':mse, 'r2':r2, 'numero de observaciones de entrenamiento':len(y_train)}

In [27]:
regresion_ridge(X,y,1)

{'rmse': 0.6097199837289877,
 'mse': 0.37175845855847706,
 'r2': 0.48195448571394395,
 'numero de observaciones de entrenamiento': 5666}

## Modelo Lasso

In [28]:
from sklearn.linear_model import Lasso

In [29]:
def regresion_lasso(X,y,alpha):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    #normalizacion de X
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # modelo ridge
    l_lasso = Lasso(alpha).fit(X_train,y_train)
    # predicciones
    y_hat = l_lasso.predict(X_test)
    # metricas
    rmse = np.sqrt(mean_squared_error(y_test,y_hat))
    mse = mean_squared_error(y_test,y_hat)
    r2 = r2_score(y_test,y_hat)
    return {'rmse':rmse, 'mse':mse, 'r2':r2, 'numero de observaciones de entrenamiento':len(y_train)}

In [30]:
regresion_lasso(X,y,0.005)

{'rmse': 0.611585434787389,
 'mse': 0.37403674404407966,
 'r2': 0.4787796942629142,
 'numero de observaciones de entrenamiento': 5666}

# Modelos de clasificacion

## Acceso a internet
Este modelo busca predecir si algun Colombiano tiene acceso a internet dadas sus caracteristicas

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [32]:
y = df1['internet']
X = df1.drop('internet', axis = 1)


In [33]:
correlacion(df1,'internet')

internet                         1.000000
television                       0.532127
computador                       0.476796
lavadora                         0.428317
educacion**2                     0.317302
educacion                        0.314611
#_cuartos_vivienda               0.297758
nevera                           0.288959
carroparticular                  0.235841
INGLABO                          0.222275
tipocontrato_Verbal              0.158479
estado_civil_Casados             0.153470
moto                             0.152343
estufa                           0.136146
tipocupacion_empleadoempresa     0.128095
tipocupacion_empleadogobierno    0.112286
seguridadsocial_asegurado        0.104092
Name: internet, dtype: float64

In [34]:
def logit_svm_random_forest(X,y,kernel_svc):
    num_variables = len(X.columns)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    #normalizacion de X
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    #modelo Logit
    lg = LogisticRegression().fit(X_train,y_train)
    #predicciones
    y_hat_logit = lg.predict(X_test)
    #metricas logit
    rmse_logit = np.sqrt(mean_squared_error(y_test,y_hat_logit))
    mse_logit = mean_squared_error(y_test,y_hat_logit)
    accuracy_logit = accuracy_score(y_test,y_hat_logit)
    logit = [rmse_logit, mse_logit,accuracy_logit,num_variables]
    # SVC
    svc = SVC(kernel = kernel_svc).fit(X_train,y_train)
    #predicciones svc
    y_hat_svc = svc.predict(X_test)
    # metricas svc
    rmse_svc = np.sqrt(mean_squared_error(y_test,y_hat_svc))
    mse_svc = mean_squared_error(y_test,y_hat_svc)
    accuracy_svc = accuracy_score(y_test,y_hat_svc)
    svmm=[rmse_svc,mse_svc,accuracy_svc,num_variables]
    # Random forest
    rf = RandomForestClassifier().fit(X_train,y_train)
    # predicciones random forest
    y_hat_rf = rf.predict(X_test)
    #metricas random forest
    rmse_rf = np.sqrt(mean_squared_error(y_test,y_hat_rf))
    mse_rf = mean_squared_error(y_test,y_hat_rf)
    accuracy_rf = accuracy_score(y_test,y_hat_rf)
    rff=[rmse_rf,mse_rf,accuracy_svc,num_variables]
    # se crea un data frame para guardar resultados
    metricas = pd.DataFrame(np.column_stack([logit, svmm, rff]), 
                               columns=['logit', 'SVC', 'Random forest'],
                            index=['rmse','mse','accuracy','numero de variables'])
   
    return metricas.T
    
    

In [35]:
logit_svm_random_forest(X,y,'rbf')


,rmse,mse,accuracy,numero de variables
logit,0.401019,0.160817,0.839183,62.0
SVC,0.396529,0.157235,0.842765,62.0
Random forest,0.384607,0.147923,0.842765,62.0


## Tenencia de vehiculo

In [36]:
correlacion(df1,'carroparticular')

carroparticular                  1.000000
INGLABO                          0.364283
educacion**2                     0.361123
computador                       0.328572
educacion                        0.313297
estado_civil_Casados             0.244289
internet                         0.235841
#_cuartos_vivienda               0.231026
lavadora                         0.219513
television                       0.195148
tipocupacion_empleadogobierno    0.181791
nevera                           0.126065
tipocupacion_empleador           0.116147
Name: carroparticular, dtype: float64

In [37]:
y = df1['carroparticular'].values
X = df1.drop('carroparticular', axis = 1)

In [38]:
logit_svm_random_forest(X,y,'rbf')

,rmse,mse,accuracy,numero de variables
logit,0.340655,0.116046,0.883954,62.0
SVC,0.343273,0.117837,0.882163,62.0
Random forest,0.333214,0.111032,0.882163,62.0


In [ ]:
a = []
for i in df['